In [1]:
import polars as pl

input_file = "data/measurements.txt"
parquet_output = "data/measurements.parquet"
summary_output = "data/summary.parquet"

lf = pl.scan_csv(
    input_file,
    separator=";",
    has_header=False,
    new_columns=["station", "temperature"],
    schema_overrides={"station": pl.Utf8, "temperature": pl.Float64},
).sink_parquet(
    parquet_output,
    row_group_size=1_000_000,
)

In [2]:
# # prepare solution
pl.scan_parquet(parquet_output).group_by("station").agg(
    [
        pl.col("temperature").min().alias("min"),
        pl.col("temperature").mean().round(2).alias("mean"),
        pl.col("temperature").max().alias("max"),
    ]
).sort("station").sink_parquet(summary_output)

In [3]:
lf = pl.read_parquet(parquet_output)
lf.head()

station,temperature
str,f64
"""Purwā""",-94.2
"""Woodburn""",-45.2
"""Pushpattūr""",65.2
"""Mŭ’minobod""",78.5
"""Buyende""",42.9


In [4]:
lf = pl.read_parquet(summary_output)
lf.head()

station,min,mean,max
str,f64,f64,f64
"""A Coruña""",-99.9,0.33,99.9
"""Aachen""",-99.9,-0.11,99.9
"""Aalen""",-99.9,0.14,99.9
"""Aarsâl""",-99.9,0.08,99.9
"""Aartselaar""",-99.9,-0.31,99.9
